## 🔑 Part 4 연습문제 정답지

#### 🎯 연습문제 1 정답

In [ ]:
gb_reg_practice = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    random_state=42
)
gb_reg_practice.fit(X_train, y_train)
y_pred_practice = gb_reg_practice.predict(X_test)
rmse_practice = np.sqrt(mean_squared_error(y_test, y_pred_practice))
print(f"변경된 하이퍼파라미터 모델의 RMSE: {rmse_practice:.4f}")

*해설: `n_estimators`를 줄이고 `learning_rate`도 낮췄기 때문에, 이전 모델보다 학습이 덜 진행되어 RMSE가 더 높게 (성능이 더 낮게) 나올 가능성이 높습니다.*

#### 🎯 연습문제 2 정답

`learning_rate`와 `n_estimators`는 일반적으로 **상충 관계(trade-off)** 에 있습니다.

  * `learning_rate`는 각 트리가 모델을 업데이트하는 보폭(step size)을 의미합니다.
  * `n_estimators`는 총 몇 걸음을 갈지(number of steps)를 의미합니다.

따라서 `learning_rate`를 낮추면(보폭을 작게 하면), 목표 성능에 도달하기 위해 더 많은 걸음, 즉 더 큰 `n_estimators`가 필요합니다. 반대로 `learning_rate`를 높이면(보폭을 크게 하면) 적은 `n_estimators`로도 빠르게 학습할 수 있지만, 최적점을 지나쳐 버릴(overshooting) 위험이 커져 과적합될 수 있습니다. 일반적으로는 `learning_rate`를 낮추고 `n_estimators`를 충분히 크게 설정한 후, 조기 종료(Early Stopping) 기법을 함께 사용하는 것이 안정적인 고성능 모델을 만드는 데 유리합니다.

#### 🎯 연습문제 3 정답

`reg_alpha`와 `reg_lambda`는 각각 L1, L2 규제를 통해 가중치(트리의 리프 노드 값)가 너무 커지는 것을 막아 모델의 복잡도를 낮추고 과적합을 방지하는 역할을 합니다. `reg_lambda=10`과 같이 큰 값을 주면 강력한 규제가 적용되어 모델이 더 단순해지며, 경우에 따라 성능이 약간 하락하거나 오히려 일반화 성능이 개선될 수 있습니다.

In [ ]:
# 문제 3 정답 코드
xgb_reg_reg = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=4,
    reg_lambda=10,  # L2 규제 강화
    random_state=42,
    n_jobs=-1
)
xgb_reg_reg.fit(X_train, y_train)
y_pred_reg = xgb_reg_reg.predict(X_test)
rmse_reg = np.sqrt(mean_squared_error(y_test, y_pred_reg))
print(f"L2 규제 적용 후 RMSE: {rmse_reg:.4f}")

#### 🎯 연습문제 4 정답

In [ ]:
# 문제 4 정답 코드
# 특성 중요도 추출
feature_importances = pd.Series(xgb_reg.feature_importances_, index=X_train.columns)
top_10_features = feature_importances.sort_values(ascending=False).head(10)

# 시각화
fig = px.bar(
    x=top_10_features.values,
    y=top_10_features.index,
    orientation='h',
    title='XGBoost Top 10 Feature Importances',
    labels={'x': 'Importance', 'y': 'Feature'}
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

*해설: 보통 시간(Hour), 온도(Temperature), 계절(Seasons)과 관련된 특성들이 상위권에 위치할 가능성이 높습니다. 특성 중요도를 통해 모델이 어떤 정보를 기반으로 예측하는지 이해할 수 있습니다.*

#### 🎯 연습문제 5 정답

In [ ]:
import time

# num_leaves = 10
start_time = time.time()
lgbm_reg_10 = lgb.LGBMRegressor(n_estimators=500, num_leaves=10, random_state=42)
lgbm_reg_10.fit(X_train, y_train)
y_pred_10 = lgbm_reg_10.predict(X_test)
rmse_10 = np.sqrt(mean_squared_error(y_test, y_pred_10))
time_10 = time.time() - start_time
print(f"num_leaves=10 | RMSE: {rmse_10:.4f}, 학습 시간: {time_10:.2f}초")

# num_leaves = 100
start_time = time.time()
lgbm_reg_100 = lgb.LGBMRegressor(n_estimators=500, num_leaves=100, random_state=42)
lgbm_reg_100.fit(X_train, y_train)
y_pred_100 = lgbm_reg_100.predict(X_test)
rmse_100 = np.sqrt(mean_squared_error(y_test, y_pred_100))
time_100 = time.time() - start_time
print(f"num_leaves=100 | RMSE: {rmse_100:.4f}, 학습 시간: {time_100:.2f}초")

*해설: `num_leaves=10`일 때는 모델이 너무 단순하여(과소적합) RMSE가 높게 나올 수 있습니다. `num_leaves=100`일 때는 모델이 더 복잡해져 RMSE는 낮아질 수 있지만(과적합 위험도 증가), 학습 시간은 더 길어집니다.*

#### 🎯 연습문제 6 정답

  * **수준 중심 분할 (Level-wise)**: XGBoost, Scikit-learn GBM 등이 사용하는 방식으로, 트리의 모든 레벨에 있는 노드를 균형적으로 분할하며 확장합니다. 즉, 같은 깊이(level)의 모든 노드를 분할한 후 다음 깊이로 넘어갑니다. 안정적이지만, 손실 감소에 별 도움이 되지 않는 노드까지 불필요하게 분할할 수 있습니다.
  * **리프 중심 분할 (Leaf-wise)**: LightGBM이 사용하는 방식으로, 현재 트리의 모든 리프 노드 중 손실(loss)을 가장 크게 줄일 수 있는 노드 하나를 선택하여 분할합니다. 비대칭적인 트리를 만들지만, 학습 초기부터 가장 효과적인 경로로 트리를 성장시키므로 수렴 속도가 빠르고 성능도 우수합니다.
  * **결과**: 리프 중심 분할은 같은 수의 노드를 가질 때 수준 중심 분할보다 더 낮은 손실을 달성할 수 있습니다. 이는 곧 더 빠른 학습 속도와 더 높은 정확도로 이어집니다. 하지만 데이터가 적을 때는 특정 경로로만 깊어지는 경향 때문에 과적합에 취약할 수 있습니다.

In [ ]:
import catboost as cb

# 원-핫 인코딩 전 데이터 다시 준비
try:
    df_orig = pd.read_csv('./datasets/SeoulBikeData.csv', encoding='cp949')
except FileNotFoundError:
    df_orig = pd.read_csv('SeoulBikeData.csv', encoding='cp949')

df_orig.columns = ['Date', 'Rented_Bike_Count', 'Hour', 'Temperature', 'Humidity',
                   'Wind_Speed', 'Visibility', 'Dew_Point_Temp', 'Solar_Radiation',
                   'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning_Day']
df_orig = df_orig.drop('Date', axis=1)

# 범주형 특성 지정
categorical_features = ['Seasons', 'Holiday', 'Functioning_Day']

# 특성과 타겟, 학습/테스트 데이터 분리
X_cat = df_orig.drop('Rented_Bike_Count', axis=1)
y_cat = df_orig['Rented_Bike_Count']
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, test_size=0.2, random_state=42)

# CatBoost 모델 초기화 및 학습
cat_reg = cb.CatBoostRegressor(
    iterations=500,         # n_estimators와 동일
    learning_rate=0.1,
    depth=6,                # max_depth와 동일
    cat_features=categorical_features, # 범주형 특성 명시
    random_state=42,
    verbose=0               # 학습 과정 출력 안 함
)
cat_reg.fit(X_train_cat, y_train_cat)

# 예측 및 성능 평가
y_pred_cat = cat_reg.predict(X_test_cat)
rmse_cat = np.sqrt(mean_squared_error(y_test_cat, y_pred_cat))
r2_cat = r2_score(y_test_cat, y_pred_cat)

print(f"\n[CatBoost Regressor 성능]")
print(f"RMSE: {rmse_cat:.4f}")
print(f"R^2 Score: {r2_cat:.4f}")

#### 🎯 연습문제 7 정답

In [ ]:
# 문제 7: cat_features 지정하지 않고 학습
cat_reg_no_cat = cb.CatBoostRegressor(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=0
)
# 범주형 변수를 pandas의 category 타입으로 바꿔주면 CatBoost가 자동으로 인식하지만,
# 여기서는 object 타입이므로 문자열을 숫자로 변환하려다 오류가 발생하거나,
# 강제로 수치형(예: 0, 1, 2...)으로 간주하게 됩니다.
# 올바른 비교를 위해, object 타입을 category 코드로 바꿔서 수치형처럼 만듭니다.
X_train_cat_no_spec = X_train_cat.copy()
X_test_cat_no_spec = X_test_cat.copy()
for col in categorical_features:
    X_train_cat_no_spec[col] = X_train_cat_no_spec[col].astype('category').cat.codes
    X_test_cat_no_spec[col] = X_test_cat_no_spec[col].astype('category').cat.codes


cat_reg_no_cat.fit(X_train_cat_no_spec, y_train_cat)
y_pred_no_cat = cat_reg_no_cat.predict(X_test_cat_no_spec)
rmse_no_cat = np.sqrt(mean_squared_error(y_test_cat, y_pred_no_cat))

print(f"cat_features 지정 O | RMSE: {rmse_cat:.4f}")
print(f"cat_features 지정 X | RMSE: {rmse_no_cat:.4f}")

*해설: `cat_features`를 지정하지 않으면 CatBoost는 'Seasons' 같은 문자열 특성을 일반적인 수치형 특성으로 취급하려고 시도합니다. 이 과정에서 'Spring' -\> 0, 'Summer' -\> 1 과 같이 임의의 순서가 부여될 수 있는데, 이는 'Summer'가 'Spring'보다 더 크다는 의미없는 수치적 관계를 모델에 주입하게 됩니다. 이는 모델 성능에 악영향을 미칩니다. `cat_features`를 지정하면 CatBoost는 Ordered TS와 같은 고급 기법을 사용하여 이러한 범주를 의미론적으로 더 잘 처리하므로 일반적으로 성능이 더 높게 나옵니다.*

#### 🎯 연습문제 8 정답

  * **원-핫 인코딩의 문제점**: '도시명' 특성에 100개의 고유 도시가 있다면, 원-핫 인코딩은 100개의 새로운 이진 특성(컬럼)을 만들어냅니다. 이는 **차원의 저주(Curse of Dimensionality)**를 유발하여 메모리 사용량이 폭증하고, 데이터가 희소(sparse)해져 모델 학습이 비효율적으로 됩니다.
  * **CatBoost의 해결책**: CatBoost는 원-핫 인코딩을 사용하지 않습니다. 대신 **Ordered Target Statistics**를 사용합니다. 100개의 도시가 있더라도, '도시명'이라는 단 하나의 특성을 유지한 채 각 도시를 해당 도시의 (이전 데이터 기준) 평균 타겟값과 같은 통계치로 대체합니다. 이 과정에서 타겟 누수를 방지하는 정교한 메커니즘을 사용하므로, 차원을 늘리지 않고도 범주형 특성의 정보를 효과적으로 모델에 전달할 수 있습니다. 이는 고유값이 수백, 수천 개인 고차원 범주형 데이터에 특히 강력한 장점입니다.


-----

## Ⅲ. 종합 실습 과제: 따릉이 수요 예측 모델 성능 비교 🏆

지금까지 배운 Gradient Boosting 모델 3대장(XGBoost, LightGBM, CatBoost)을 모두 사용하여 '서울 따릉이' 수요 예측 문제를 해결해보고, 최종 성능을 비교 분석하여 최적의 모델을 찾아봅시다.

### 과제 목표

1.  간단한 특성 공학(Feature Engineering)을 수행합니다.
2.  XGBoost, LightGBM, CatBoost 모델을 각각 학습시키고 성능(RMSE, R²)을 평가합니다.
3.  세 모델의 성능을 시각적으로 비교하고 결과를 해석합니다.

### Step 1: 데이터 준비 및 특성 공학

기존에 사용한 `df_orig` 데이터프레임에 새로운 시간 관련 특성을 추가해 봅시다.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import catboost as cb


df_final = pd.read_csv('./datasets/SeoulBikeData.csv', encoding='cp949')

df_final.columns = ['Date', 'Rented_Bike_Count', 'Hour', 'Temperature', 'Humidity',
                    'Wind_Speed', 'Visibility', 'Dew_Point_Temp', 'Solar_Radiation',
                    'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning_Day']

In [ ]:
# 원본 데이터 로드
df_final = pd.read_csv('../datasets/ml/bike-sharing/SeoulBikeData.csv', encoding='cp949')

# 컬럼명 정리
df_final.columns = ['Date', 'Rented_Bike_Count', 'Hour', 'Temperature', 'Humidity',
                    'Wind_Speed', 'Visibility', 'Dew_Point_Temp', 'Solar_Radiation',
                    'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning_Day']

# 📌 특성 공학: 주중/주말, 출퇴근 시간 특성 추가
df_final['Date'] = pd.to_datetime(df_final['Date'], format='%d/%m/%Y')
df_final['Weekday'] = df_final['Date'].dt.weekday # 0:월, 6:일
df_final['Is_Weekend'] = df_final['Weekday'].apply(lambda x: 1 if x >= 5 else 0)
df_final['Is_Rush_Hour'] = df_final['Hour'].apply(lambda x: 1 if (x >= 7 and x <= 9) or (x >= 17 and x <= 19) else 0)

# 사용할 특성 정의
features = ['Hour', 'Temperature', 'Humidity', 'Wind_Speed', 'Visibility',
            'Solar_Radiation', 'Rainfall', 'Snowfall', 'Seasons', 'Holiday',
            'Functioning_Day', 'Is_Weekend', 'Is_Rush_Hour']
target = 'Rented_Bike_Count'

# 범주형 특성 목록
categorical_features_final = ['Seasons', 'Holiday', 'Functioning_Day']

X_final = df_final[features]
y_final = df_final[target]

# 학습/테스트 데이터 분리
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

### Step 2: 3개 모델 학습 및 평가

아래 가이드에 따라 XGBoost, LightGBM, CatBoost 모델을 각각 학습시키고, 결과를 `results` 리스트에 저장하세요.

  * **XGBoost**: `pd.get_dummies`로 범주형 변수를 원-핫 인코딩한 데이터를 사용해야 합니다.
  * **LightGBM**: `pd.get_dummies`로 인코딩된 데이터를 사용하거나, `pandas.Categorical` 타입으로 변환하여 직접 처리할 수 있습니다. 여기서는 XGBoost와 동일하게 인코딩된 데이터를 사용하겠습니다.
  * **CatBoost**: 원-핫 인코딩되지 않은 원본 형태의 데이터를 사용하고, `cat_features` 인자를 꼭 설정해주세요.

<!-- end list -->

In [ ]:
# 결과 저장을 위한 리스트
results = []

# --- 1. XGBoost ---
# XGBoost를 위한 데이터 전처리 (원-핫 인코딩)
X_train_xgb_f = pd.get_dummies(X_train_f, columns=categorical_features_final, drop_first=True)
X_test_xgb_f = pd.get_dummies(X_test_f, columns=categorical_features_final, drop_first=True)

# 모델 학습
xgb_final = xgb.XGBRegressor(random_state=42, n_jobs=-1)
xgb_final.fit(X_train_xgb_f, y_train_f)
y_pred_xgb_f = xgb_final.predict(X_test_xgb_f)

# 성능 계산 및 저장
rmse_xgb_f = np.sqrt(mean_squared_error(y_test_f, y_pred_xgb_f))
r2_xgb_f = r2_score(y_test_f, y_pred_xgb_f)
results.append({'Model': 'XGBoost', 'RMSE': rmse_xgb_f, 'R2': r2_xgb_f})


# --- 2. LightGBM ---
# LightGBM은 XGBoost와 동일한 데이터 사용
lgbm_final = lgb.LGBMRegressor(random_state=42, n_jobs=-1)
lgbm_final.fit(X_train_xgb_f, y_train_f) # XGBoost용으로 인코딩된 데이터 사용
y_pred_lgbm_f = lgbm_final.predict(X_test_xgb_f)

# 성능 계산 및 저장
rmse_lgbm_f = np.sqrt(mean_squared_error(y_test_f, y_pred_lgbm_f))
r2_lgbm_f = r2_score(y_test_f, y_pred_lgbm_f)
results.append({'Model': 'LightGBM', 'RMSE': rmse_lgbm_f, 'R2': r2_lgbm_f})


# --- 3. CatBoost ---
# CatBoost는 원본 데이터 사용
cat_final = cb.CatBoostRegressor(random_state=42, cat_features=categorical_features_final, verbose=0)
cat_final.fit(X_train_f, y_train_f) # 인코딩 전 데이터 사용
y_pred_cat_f = cat_final.predict(X_test_f)

# 성능 계산 및 저장
rmse_cat_f = np.sqrt(mean_squared_error(y_test_f, y_pred_cat_f))
r2_cat_f = r2_score(y_test_f, y_pred_cat_f)
results.append({'Model': 'CatBoost', 'RMSE': rmse_cat_f, 'R2': r2_cat_f})

# 결과 데이터프레임 생성
results_df = pd.DataFrame(results)
print(results_df)

### Step 3: 결과 비교 및 시각화

`plotly.express`를 사용하여 세 모델의 RMSE와 R² 점수를 막대그래프로 비교해 봅시다.

In [ ]:
# RMSE 비교 시각화
fig_rmse = px.bar(results_df.sort_values('RMSE'),
                  x='Model',
                  y='RMSE',
                  title='Model Comparison: RMSE (Lower is Better)',
                  color='Model',
                  text_auto='.4f')
fig_rmse.show()

# R² Score 비교 시각화
fig_r2 = px.bar(results_df.sort_values('R2', ascending=False),
                x='Model',
                y='R2',
                title='Model Comparison: R² Score (Higher is Better)',
                color='Model',
                text_auto='.4f')
fig_r2.show()

### Step 4: 결과 해석 (자율 양식)

  * 어떤 모델이 가장 좋은 성능(가장 낮은 RMSE, 가장 높은 R²)을 보였나요?
  * 각 모델의 학습 시간은 어땠나요? (코드를 수정하여 측정해보세요.)
  * 이 데이터셋의 특성(범주형 변수의 존재, 데이터 크기 등)을 고려할 때, 왜 특정 모델이 더 좋은 성능을 보였을지 자신의 생각을 정리해보세요.
  * 기타 생각해볼 점이 있다면 적어보세요.